# **Notebook Completo para Reconocimiento Facial (Colab + Firebase + Flutter)**
## Celda 1: Configuración Inicial e Instalación de Dependencias
Propósito: Esta celda prepara el entorno. Primero, subes tu clave de Firebase para que el script pueda acceder a tus datos. Luego, instala todas las librerías necesarias (firebase-admin para conectar con Firebase, deepface para el reconocimiento facial y pyngrok para crear la API pública).

In [3]:
# ==============================================================================
# Celda 1: Configuración e Instalación
# ==============================================================================
# Propósito: Preparar el entorno de Colab, subir la clave de Firebase
# e instalar todas las librerías necesarias.
# ==============================================================================

# Paso 1: Subir la clave de servicio de Firebase
# ------------------------------------------------------------------------------
# Al ejecutar esta celda, aparecerá un botón para que subas el archivo .json
# que descargaste desde la configuración de tu proyecto de Firebase.
# Este archivo es esencial para que el script se autentique de forma segura.

from google.colab import files

print("Por favor, sube el archivo JSON de tu cuenta de servicio de Firebase.")
uploaded = files.upload()

# Guardamos el nombre del archivo subido para usarlo más adelante
key_filename = list(uploaded.keys())[0]
print(f"\n✅ Archivo '{key_filename}' subido correctamente.")


# Paso 2: Instalar las librerías de Python necesarias
# ------------------------------------------------------------------------------
# - firebase-admin: La librería oficial de Google para interactuar con Firebase.
# - deepface: Una potente librería que simplifica el reconocimiento facial.
# - pyngrok: Herramienta para crear una URL pública y conectar Flutter con Colab.
# - opencv-python-headless, numpy, Pillow: Librerías para el análisis de imágenes
#   y la detección de spoofing (fotos de fotos).

print("\nInstalando dependencias... (Esto puede tardar un par de minutos)")
!pip install firebase-admin deepface pyngrok opencv-python-headless numpy Pillow --quiet
print("✅ Dependencias instaladas.")

Por favor, sube el archivo JSON de tu cuenta de servicio de Firebase.


Saving geoface-c53eb-firebase-adminsdk-fbsvc-dbb0ac7ff5.json to geoface-c53eb-firebase-adminsdk-fbsvc-dbb0ac7ff5.json

✅ Archivo 'geoface-c53eb-firebase-adminsdk-fbsvc-dbb0ac7ff5.json' subido correctamente.

Instalando dependencias... (Esto puede tardar un par de minutos)
✅ Dependencias instaladas.


## Celda 2: Lógica Principal (Conexión a Firebase y Funciones de IA)
### Propósito: Aquí reside el "cerebro" de nuestro sistema.
1. Inicializa Firebase: Usa la clave que subiste para conectar con tu proyecto.
2. Sincroniza la Base de Datos Facial: Lee la colección biometricos de tu Firestore, descarga cada foto de empleado desde Cloud Storage y la organiza en una estructura de carpetas local (DB_PATH/empleadoId/foto.jpg). DeepFace usará esta estructura para saber a quién pertenece cada rostro.
3. Define la Función de Identificación: Crea la función identificar_empleado que toma una imagen nueva (la que enviará tu app) y la compara contra la base de datos local para encontrar la mejor coincidencia y devolver el empleadoId.

In [ ]:
# ==============================================================================
# Celda 2: Lógica Principal - Conexión a Firebase y Funciones de IA (Versión Mejorada con Anti-Spoofing Avanzado)
# ==============================================================================
# Propósito: Conectar con Firebase, descargar fotos, y definir funciones de IA
# que incluyen un robusto sistema anti-spoofing antes del reconocimiento facial.
# ==============================================================================

import os
import shutil
import requests
import firebase_admin
from firebase_admin import credentials, firestore
from deepface import DeepFace
# --- NUEVAS LIBRERÍAS NECESARIAS PARA ANTI-SPOOFING ---
import cv2
import numpy as np
from PIL import Image, ImageStat

# --- CONFIGURACIÓN GLOBAL ---
SERVICE_ACCOUNT_KEY = key_filename
DB_PATH = "employee_face_db"

# --- FUNCIONES DE INFRAESTRUCTURA (Firebase y Sincronización) ---

def initialize_firebase():
    """Inicializa la app de Firebase usando la clave de servicio."""
    try:
        if not firebase_admin._apps:
            cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
            firebase_admin.initialize_app(cred)
            print("✅ Conexión con Firebase establecida correctamente.")
        else:
            print("ℹ️ Firebase App ya estaba inicializada.")
    except Exception as e:
        print(f"❌ Error al inicializar Firebase: {e}")
        raise


def sync_face_database_from_firestore():
    print("\n🔄 Sincronizando base de datos...")
    if os.path.exists(DB_PATH):
        shutil.rmtree(DB_PATH)
    os.makedirs(DB_PATH, exist_ok=True)
    db = firestore.client()
    docs = db.collection('biometricos').stream()
    image_count = 0
    employee_folders = set()
    
    for doc in docs:
        data = doc.to_dict()
        empleado_id = data.get('empleadoId')
        # CORRECCIÓN: Buscar 'datosFaciales' (plural, array) en lugar de 'datoFacial' (singular)
        datos_faciales = data.get('datosFaciales', [])
        
        if not empleado_id or not datos_faciales:
            if not empleado_id:
                print(f"   ⚠️  Documento {doc.id} sin empleadoId, omitiendo...")
            elif not datos_faciales:
                print(f"   ⚠️  Empleado {empleado_id} sin datosFaciales, omitiendo...")
            continue
        
        # Crear directorio para el empleado
        employee_dir = os.path.join(DB_PATH, empleado_id)
        os.makedirs(employee_dir, exist_ok=True)
        employee_folders.add(empleado_id)
        
        # CORRECCIÓN: Iterar sobre todas las URLs del array datosFaciales
        if isinstance(datos_faciales, list):
            for idx, image_url in enumerate(datos_faciales):
                if not image_url or not isinstance(image_url, str):
                    continue
                try:
                    response = requests.get(image_url, timeout=10)
                    if response.status_code == 200:
                        # Guardar cada imagen con un nombre único (índice + UUID del documento)
                        filename = f"{doc.id}_{idx}.jpg"
                        filepath = os.path.join(employee_dir, filename)
                        with open(filepath, 'wb') as f:
                            f.write(response.content)
                        image_count += 1
                        print(f"   ✅ Descargada imagen {idx+1}/{len(datos_faciales)} para empleado {empleado_id}")
                    else:
                        print(f"   ⚠️  Error HTTP {response.status_code} al descargar {image_url}")
                except Exception as e:
                    print(f"   ⚠️  Excepción al descargar imagen {idx+1} de {empleado_id}: {e}")
        else:
            print(f"   ⚠️  datosFaciales no es un array para empleado {empleado_id}, tipo: {type(datos_faciales)}")
    
    print(f"\n✅ Sincronización completada: {image_count} imágenes para {len(employee_folders)} empleados.")
    
    # Eliminar cache de DeepFace si existe para forzar regeneración
    cache_file = os.path.join(DB_PATH, "representations_arcface.pkl")
    if os.path.exists(cache_file):
        os.remove(cache_file)
        print("ℹ️ Cache (.pkl) eliminado para forzar regeneración con nuevas imágenes.")


# --- NUEVAS FUNCIONES ANTI-SPOOFING (Análisis de Imagen Completa) ---

def detectar_foto_de_foto(ruta_imagen):
    """
    Detecta si una imagen es una foto de una pantalla o papel usando análisis de imagen.
    """
    try:
        # Cargar la imagen con OpenCV
        img_cv = cv2.imread(ruta_imagen)
        if img_cv is None:
            return False, {"error": "No se pudo cargar la imagen con OpenCV."}

        gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

        # Puntuación y razones para la decisión
        score_spoofing = 0
        razones = []

        # TÉCNICA 1: Detección de patrones de Moiré (comunes en fotos de pantallas)
        f_transform = np.fft.fft2(gray)
        f_shift = np.fft.fftshift(f_transform)
        magnitude_spectrum = 20 * np.log(np.abs(f_shift))
        # Los patrones de Moiré crean picos de alta frecuencia muy marcados
        picos_altos = np.sum(magnitude_spectrum > np.mean(magnitude_spectrum) * 1.5)
        if picos_altos > (gray.shape[0] * gray.shape[1] * 0.01): # Si más del 1% de los puntos son picos
            score_spoofing += 35
            razones.append("Patrones de alta frecuencia detectados (posible pantalla)")

        # TÉCNICA 2: Análisis de reflejos y brillo
        _, thresh = cv2.threshold(gray, 245, 255, cv2.THRESH_BINARY)
        pixeles_brillantes = cv2.countNonZero(thresh)
        porcentaje_brillo = (pixeles_brillantes / gray.size) * 100
        if porcentaje_brillo > 0.5: # Incluso un 0.5% de reflejo puro es sospechoso
             score_spoofing += 25
             razones.append(f"Reflejos o áreas sobreexpuestas detectadas ({porcentaje_brillo:.2f}%)")

        # TÉCNICA 3: Varianza de color (las fotos de fotos suelen ser más planas)
        pil_img = Image.open(ruta_imagen)
        stat = ImageStat.Stat(pil_img)
        varianza_promedio = sum(stat.var) / len(stat.var) if len(stat.var) > 0 else 0
        if varianza_promedio < 1500: # Las fotos reales suelen tener alta varianza
            score_spoofing += 20
            razones.append(f"Baja varianza de color ({varianza_promedio:.1f})")

        # TÉCNICA 4: Detección de bordes rectos (marcos de teléfono o papel)
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=80, minLineLength=100, maxLineGap=10)
        if lines is not None and len(lines) > 2:
            score_spoofing += 20
            razones.append(f"Bordes rectos detectados ({len(lines)}), posible marco de pantalla")

        # Decisión final
        es_spoofing = score_spoofing >= 65 # Umbral de decisión (ajustable)

        detalles = {
            "score": score_spoofing,
            "es_spoofing": es_spoofing,
            "razones": razones if es_spoofing else ["Imagen parece auténtica"]
        }

        return es_spoofing, detalles

    except Exception as e:
        return False, {"error": f"Excepción en análisis de spoofing: {e}"}


# --- FUNCIÓN PRINCIPAL DE IDENTIFICACIÓN ---
# Esta es la función que será llamada por el servidor Flask.

def identificar_empleado(ruta_imagen_a_verificar, ruta_db):
    """
    Función de identificación robusta:
    1. Ejecuta un análisis anti-spoofing sobre la imagen completa.
    2. Si la imagen es auténtica, procede con el reconocimiento facial usando DeepFace.
    """
    print("\n📸 Iniciando proceso de identificación...")

    # PASO 1: Análisis anti-spoofing de la imagen completa
    # ----------------------------------------------------
    print("   - (1/2) Realizando análisis de autenticidad de la imagen...")
    es_spoofing, detalles_spoofing = detectar_foto_de_foto(ruta_imagen_a_verificar)

    if es_spoofing:
        razones_str = "; ".join(detalles_spoofing.get("razones", []))
        score = detalles_spoofing.get('score', 'N/A')
        print(f"   - 🚫 SPOOFING DETECTADO (Score: {score}). Razones: {razones_str}")
        return f"Error: Foto Falsa Detectada. {razones_str}"

    score = detalles_spoofing.get('score', 'N/A')
    print(f"   - ✅ Imagen parece auténtica (Score de spoofing: {score}).")

    # PASO 2: Reconocimiento Facial
    # -----------------------------
    print("   - (2/2) Realizando reconocimiento facial...")
    try:
        # Usamos DeepFace.find para la identificación 1-vs-N
        dfs = DeepFace.find(
            img_path=ruta_imagen_a_verificar,
            db_path=ruta_db,
            model_name='ArcFace',
            detector_backend='retinaface',
            enforce_detection=True,
            silent=True
        )

        # Si no se encuentra ninguna coincidencia
        if not dfs or dfs[0].empty:
            print("   - ❌ Resultado: Desconocido. No se encontró ninguna coincidencia en la BD.")
            return "Desconocido"

        # Se encontró una coincidencia, devolvemos el ID del empleado
        identidad = dfs[0].iloc[0]
        empleado_id = os.path.basename(os.path.dirname(identidad['identity']))
        distancia = identidad['distance']

        print(f"   - ✅ Resultado: Empleado identificado - ID: {empleado_id} (Distancia: {distancia:.3f})")
        return empleado_id

    except ValueError as e:
        # Error común si DeepFace no detecta una cara
        if "Face could not be detected" in str(e):
            print("   - ❌ Error de DeepFace: No se detectó un rostro claro.")
            return "Error: No se detectó un rostro claro en la imagen."
        print(f"   - ❌ Error de DeepFace: {e}")
        return f"Error en DeepFace: {e}"
    except Exception as e:
        print(f"   - ❌ Excepción inesperada en DeepFace: {e}")
        return f"Error inesperado durante el reconocimiento: {e}"


# --- EJECUCIÓN DEL PROCESO DE PREPARACIÓN ---
initialize_firebase()
sync_face_database_from_firestore()

ℹ️ Firebase App ya estaba inicializada.

🔄 Sincronizando base de datos...
✅ Sincronización completada: 0 imágenes para 4 empleados.


## **Celda 3: Iniciar el Servidor API**
### Propósito: Esta es la última celda. Pone en marcha un servidor web (usando Flask) que expone nuestra función identificar_empleado al mundo exterior.
1. Configura pyngrok: Pide tu Authtoken para poder crear la URL pública.
2. Define el Endpoint /identificar: Esta es la dirección específica a la que tu app de Flutter hará la llamada. Está programada para recibir un archivo de imagen (face_image).
3. Inicia el Túnel y el Servidor: Crea la URL pública con pyngrok y la imprime en pantalla. Luego, arranca el servidor Flask, que se quedará esperando las peticiones de tu app. La ejecución se quedará "colgada" en esta celda mientras el servidor esté activo, lo cual es normal.

In [2]:
# ==============================================================================
# Celda 3: Iniciar el Servidor API con Flask y Ngrok
# ==============================================================================
# Propósito: Levantar un servidor web que tu app de Flutter pueda consumir.
# Este servidor tendrá un endpoint (/identificar) que recibe una imagen,
# la procesa con la función de IA (que incluye anti-spoofing) y devuelve el resultado.
# ==============================================================================

from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
import uuid
import os

# --- CONFIGURACIÓN DE NGROK ---
# Pega aquí tu Authtoken de Ngrok. Lo obtienes desde tu dashboard de ngrok.com
NGROK_AUTH_TOKEN = "2vVPYuy7nBsvhhxY2bTTjryatYH_3MVU7P1sQFUYKqcx1JJNh"

if not NGROK_AUTH_TOKEN or "PEGA_AQUI" in NGROK_AUTH_TOKEN:
    print("❌ ERROR: Por favor, edita esta celda y pega tu Authtoken de Ngrok.")
else:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN

    # --- CONFIGURACIÓN DEL SERVIDOR FLASK ---
    app = Flask(__name__)

    # Carpeta para guardar temporalmente las imágenes recibidas
    uploads_dir = "uploads"
    os.makedirs(uploads_dir, exist_ok=True)

    # Definir el endpoint (la URL específica que llamará Flutter)
    @app.route('/identificar', methods=['POST'])
    def endpoint_identificar():
        if 'face_image' not in request.files:
            return jsonify({"error": "No se envió ningún archivo de imagen.", "empleadoId": None}), 400

        file = request.files['face_image']
        if not file or file.filename == '':
            return jsonify({"error": "Archivo inválido o sin nombre.", "empleadoId": None}), 400

        # Guardar el archivo temporalmente
        filename = str(uuid.uuid4()) + os.path.splitext(file.filename)[1]
        filepath = os.path.join(uploads_dir, filename)
        file.save(filepath)

        # --- CAMBIO CLAVE AQUÍ ---
        # Llamar a la función de reconocimiento facial principal de la Celda 2
        resultado_id = identificar_empleado(filepath, DB_PATH)

        # Borrar el archivo temporal
        os.remove(filepath)

        # Devolver el resultado en formato JSON
        # Esta lógica ahora maneja los nuevos mensajes de error de forma más explícita
        if "Error" in resultado_id or "Desconocido" in resultado_id:
            # Los logs ya se imprimen dentro de la función `identificar_empleado`
            # Devolvemos un código 403 (Prohibido) si detectamos spoofing, 404 para otros errores.
            status_code = 403 if "Foto Falsa Detectada" in resultado_id else 404
            return jsonify({"error": resultado_id, "empleadoId": None}), status_code
        else:
            # Los logs de éxito ya se imprimen dentro de la función
            return jsonify({"empleadoId": resultado_id, "error": None}), 200

    @app.route('/sync-database', methods=['POST'])
    def endpoint_sync():
        print("\n🔄 Solicitud de sincronización manual recibida...")
        try:
            sync_face_database_from_firestore()
            return jsonify({"message": "Sincronización completada."}), 200
        except Exception as e:
            return jsonify({"error": f"Error al sincronizar: {e}"}), 500

    # --- INICIAR EL SERVIDOR Y EL TÚNEL ---
    # Abre el túnel de ngrok hacia el puerto 5000 (puerto por defecto de Flask)
    public_url = ngrok.connect(5000, "http")
    print("=" * 60)
    print("✅ ¡API LISTA PARA USAR!")
    print(f"   Tu URL pública para Flutter es: {public_url}")
    print("   Recuerda añadir '/identificar' al final en tu código de Flutter.")
    print("=" * 60)

    # Inicia la app de Flask. Se quedará corriendo hasta que detengas la celda.
    app.run()

✅ ¡API LISTA PARA USAR!
   Tu URL pública para Flutter es: NgrokTunnel: "https://c8a3aa65e9f9.ngrok-free.app" -> "http://localhost:5000"
   Recuerda añadir '/identificar' al final en tu código de Flutter.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 01:44:22] "POST /identificar HTTP/1.1" 404 -



📸 Iniciando proceso de identificación...
   - (1/2) Realizando análisis de autenticidad de la imagen...
   - ✅ Imagen parece auténtica (Score de spoofing: 35).
   - (2/2) Realizando reconocimiento facial...
   - ❌ Error de DeepFace: No item found in employee_face_db


INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 01:44:31] "POST /identificar HTTP/1.1" 404 -



📸 Iniciando proceso de identificación...
   - (1/2) Realizando análisis de autenticidad de la imagen...
   - ✅ Imagen parece auténtica (Score de spoofing: 35).
   - (2/2) Realizando reconocimiento facial...
   - ❌ Error de DeepFace: No item found in employee_face_db


...

INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 01:49:13] "POST /sync-database HTTP/1.1" 200 -


✅ Sincronización completada: 0 imágenes para 4 empleados.

🔄 Solicitud de sincronización manual recibida...

🔄 Sincronizando base de datos...


INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 01:50:38] "POST /sync-database HTTP/1.1" 200 -


✅ Sincronización completada: 0 imágenes para 4 empleados.


KeyboardInterrupt: 